This notebook will introduce the functionalities in the Neural Simulated Annealing code base to the best of our current knowledge.

Unless otherwise stated, information comes from either the [paper](../references/neural-simulated-annealing.pdf) or directly from reading the [source code](https://github.com/tomaz-suller/neural-simulated-annealing)

Neural Simulated Annealing (NeuralSA) works by employing a reinforcement learning agent to solve combinatorial search problems, with multiple example implementations provided in the source code.

Agents are deep reinforcement learning models implemented in PyTorch

In [2]:
import torch
from torch import Tensor

# Initial Attempt: NATS-Bench specific

## Environment Definition

The environment is defined by an instance of the `neuralsa.problem.Problem` abstract class

In [3]:
from neuralsa.problem import Problem

### Data model

Most instance variables are `torch.Tensor`s since a single problem instance is actually meant to describe multiple instantiations of the same problem to be solved in parallel.

#### Parameters

The class stores the current state of the problem and parameters (`params`) which define the problem (e.g. the weights and values of items in the Knapsack problem, and the capacity of the knapsack).

The NAS problem could possibly be described using these parameters in a generic sense, but this first implementation will only deal with the specific case of the NATS-Bench Topology benchmark.

#### State

The state is described by a column vector or matrix which is returned by the `state_encoding` property.

One sneaky detail of the current implementation is the `x_dim` integer static class variable, which describes the number of dimensions in `x` (which should be renamed ASAP and which I _believe_ to be the current problem solution).

I haven't quite figured out why the current SA temperature is also stored in the state matrix...

### How to implement functions?

Some functions are clearly named using reinforcement learning jargon (e.g. `cost`, `update`)

The `main.py` script in the NeuralSA repo hints at the roles of some less clearly named functions in `Problem`:

| Function | Role |
|----------|------|
| `generate_params` | Returns a random instance of the problem parameters |
| `set_params` | Sets problem parameters for the `Problem` instance | 
| `state_encoding` | Column vector or matrix representing the current state variables of the problem |


### Implementation

To avoid magic numbers, we will use some definitions in the NATS-Bench wrapper.

In [4]:
from nas.benchmark.nats_bench import Operation, CellTopology

In [5]:
len(Operation), CellTopology.number_operations()

(5, 6)

The __state__ will be a tensor of dimensions `[n_problems, len(Operation), CellTopology.number_operations()]` which will use one-hot encoding to represent operations (and thus avoid any undue correlation which using e.g. sequential numbers would bring).

An __action__ will be a tensor with the same shape as the current state, and will contain a single elements with value 1, representing the cell operations which should be set.

The NAS problem does not seem at first glance to require any state apart from the solution, which implies that the return value of the `state_encoding` function is meaningless. Ideally it would be null, but that would not be supported by PyTorch most likely.

In [1]:
class NatsBenchTopology(Problem):
    x_dim = 6  # 6 edges to select in the cell

    def cost(self, s: torch.Tensor) -> torch.float:
        # TODO Get model results from NATS-Bench
        ...

    def update(self, s: Tensor, a: Tensor) -> Tensor: ...

    def set_params(self, **kwargs) -> None: ...

    def generate_params(self) -> dict[str, Tensor]:
        return {}

    @property
    def state_encoding(self) -> torch.Tensor: ...

    def generate_init_state(self) -> Tensor:
        return super().generate_init_state()

IndentationError: expected an indented block after function definition on line 4 (4130900577.py, line 7)